# Optimization Visualization

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.colors import LogNorm
from IPython.display import HTML
from math import sqrt
from math import pi
%matplotlib inline

## Cost Funcitons

In [2]:
def Rad(deg):
    return deg * pi / 180

def Rt(x, y, th):
    R = np.array([[np.cos(th), -np.sin(th)], [np.sin(th), np.cos(th)]])
    t = np.array([[x], [y]])
    return R, t

def LS(cost):
    return 0.5 * cost ** 2

def ICP(p, q, x, y, th):
    R, t = Rt(x, y, th)
    cost = np.linalg.norm(R.dot(p) + t - q)
    return LS(cost)

def NICP(p, q, nq, x, y, th):
    R, t = Rt(x, y, th)
    cost = (R.dot(p) + t - q).T.dot(nq)[0, 0]
    return LS(cost)

def SICP(p, np, q, nq, x, y, th):
    R, t = Rt(x, y, th)
    if (np.T.dot(nq)[0, 0] < 0):
        np = -np
    cost = (R.dot(p) + t - q).T.dot(np + nq)[0, 0]
    return LS(cost)

def P2D(up, uq, cq, x, y, th):
    R, t = Rt(x, y, th)
    m = R.dot(up) + t - uq
    cost = sqrt(m.T.dot(np.linalg.inv(cq).dot(m))[0, 0])
    return LS(cost)

def D2D(up, cp, uq, cq, x, y, th):
    R, t = Rt(x, y, th)
    m = R.dot(up) + t - uq
    c = R.dot(cp).dot(R.T) + cq
    cost = sqrt(m.T.dot(np.linalg.inv(c).dot(m))[0, 0])
    return LS(cost)

def SNDT(up, cp, unp, cnp, uq, cq, unq, cnq, x, y, th):
    R = np.array([[np.cos(th), -np.sin(th)], [np.sin(th), np.cos(th)]])
    t = np.array([[x], [y]])
    if (unp.T.dot(unq)[0, 0] < 0):
        unp = -unp
    m1 = R.dot(up) + t - uq
    m2 = R.dot(unp) + unq
    c1 = R.dot(cp).dot(R.T) + cq
    c2 = R.dot(cnp).dot(R.T) + cnq
    mu = m1.T.dot(m2)[0, 0]
    std = sqrt(m1.T.dot(c2.dot(m1)) + m2.T.dot(c1.dot(m2)) + np.trace(c1.dot(c2)))
    cost = mu / std
    return LS(cost)

def SNDT2(up, cp, unp, uq, cq, unq, x, y, th):
    R = np.array([[np.cos(th), -np.sin(th)], [np.sin(th), np.cos(th)]])
    t = np.array([[x], [y]])
    if (unp.T.dot(unq)[0, 0] < 0):
        unp = -unp
    m1 = R.dot(up) + t - uq
    m2 = R.dot(unp) + unq
    c1 = R.dot(cp).dot(R.T) + cq
    mu = m1.T.dot(m2)[0, 0]
    std = sqrt(m2.T.dot(c1.dot(m2)))
    cost = mu / std
    return LS(cost)

## Data

In [3]:
up = np.array([[-30.638454], [64.600220]])
cp = np.array([[0.191993, 0.177331], [0.177331, 0.170498]])
unp = np.array([[-0.597271], [0.790676]])
cnp = np.array([[0.013271, 0.009066], [0.009066, 0.006337]])
uq = np.array([[-16.883654], [58.616450]])
cq = np.array([[0.191993, 0.177331], [0.177331, 0.170498]])
unq = np.array([[-0.597271], [0.790676]])
cnq = np.array([[0.013271, 0.009066], [0.009066, 0.006337]])
gt = np.array([[13.7548], [-5.98377]])

## Contour Helper Function

In [4]:
def Contour(x, y, m, gt):
    n = len(x)

    off = 5
    xmax = max(abs(min(x) - off), abs(max(x) + off))
    xmin = -xmax
    ymax = max(abs(min(y) - off), abs(max(y) + off))
    ymin = -ymax

    xx = np.linspace(xmin, xmax, 100)
    yy = np.linspace(ymin, ymax, 100)
    xg, yg = np.meshgrid(xx, yy)
    zg = np.zeros(xg.shape)

    %matplotlib
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(*gt, 'r*', markersize=18, label='Ground Truth')
    ax.set_xlim((xmin, xmax))
    ax.set_ylim((ymin, ymax))
    ax.set_xlabel('x [m]')
    ax.set_ylabel('y [m]')

    for i in range(xg.shape[0]):
        for j in range(xg.shape[1]):
            if (m == 'ICP'):
                zg[i, j] = ICP(up, uq, xg[i, j], yg[i, j], 0)
            elif (m == 'NICP'):
                zg[i, j] = NICP(up, uq, unq, xg[i, j], yg[i, j], 0)
            elif (m == 'SICP'):
                zg[i, j] = SICP(up, unp, uq, unq, xg[i, j], yg[i, j], 0)
            elif (m == 'P2D'):
                zg[i, j] = P2D(up, uq, cq, xg[i, j], yg[i, j], 0)
            elif (m == 'D2D'):
                zg[i, j] = D2D(up, cp, uq, cq, xg[i, j], yg[i, j], 0)
            elif (m == 'SNDT'):
                zg[i, j] = SNDT(up, cp, unp, cnp, uq, cq, unq, cnq, xg[i, j], yg[i, j], 0)
            elif (m == 'SNDT2'):
                zg[i, j] = SNDT2(up, cp, unp, uq, cq, unq, xg[i, j], yg[i, j], 0)
    ax.contour(xg, yg, zg, levels=np.logspace(0, 5, 20), norm=LogNorm(), cmap=plt.cm.jet)
    
    line, = ax.plot([], [], 'r', label=m + ' Algorithm')
    point, = ax.plot([], [], 'ro')
    ax.legend()

    %matplotlib inline
    def init():
        return line, point

    def update(i):
        line.set_data(x[:i], y[:i])
        point.set_data(x[i - 1], y[i - 1])
        return line, point

    ani = FuncAnimation(fig, update, frames=n, init_func=init, interval=150, repeat_delay=5, blit=True)
    return HTML(ani.to_html5_video())

## ICP

In [5]:
x = [0, 13.7534, 13.7548, 13.7548, 13.7548, ]
y = [0, -5.98317, -5.98377, -5.98377, -5.98377, ]
Contour(x, y, 'ICP', gt)

Using matplotlib backend: Qt5Agg


## Point-to-Plane ICP

In [6]:
x = [0, 10.8376, 10.8381, 10.8381, 10.8381, ]
y = [0, -8.18661, -8.18702, -8.18702, -8.18702, ]
Contour(x, y, 'NICP', gt)

Using matplotlib backend: Qt5Agg


## Symmetric ICP

In [7]:
x = [0, 10.8376, 10.8381, 10.8381, 10.8381, ]
y = [0, -8.18661, -8.18702, -8.18702, -8.18702, ]
Contour(x, y, 'SICP', gt)

Using matplotlib backend: Qt5Agg


## Point-to-Distribution NDT

In [8]:
x = [0, 10.0284, 10.0284, 10.0284, 10.0284, 10.0284, 10.0284, 10.0284, 10.1899, 10.1899, 10.1899, 10.1607, 10.1607, 10.1607, 10.1607, 10.1544, 10.1601, 10.1601, 10.1555, 10.1594, 10.1594, 10.1559, 10.158, 10.1588, 10.1588, 10.1588, 10.1568, 10.1582, 10.1582, 10.1587, 10.1587, 10.1587, 10.1569, 10.1589, 10.1562, 10.1573, 10.1608, 10.1595, 10.1552, 10.156, 10.1594, 10.1594, 10.1594, 10.1582, 10.1587, 10.158, 10.1606, 10.1592, 10.1592, 10.1572, 10.1586, 10.1585, 10.1599, 10.1599, 10.1557, 10.157, 10.1616, 10.159, 10.1562, 10.1594, 10.1594, 10.1594, 10.1585, 10.1594, 10.1583, 10.1598, 10.158, 10.1596, 10.1581, 10.1598, 10.1579, 10.1593, 10.1586, 10.1604, 10.1584, 10.1612, 10.1603, 10.1571, 10.1589, 10.1608, 10.1582, 10.1582, 10.1582, 10.1582, 10.1582, 10.1582, 10.1582, 10.1582, 10.1582, 10.1582, ]
y = [0, -9.52049, -9.52049, -9.52049, -9.52049, -9.52049, -9.52049, -9.52049, -9.44805, -9.44805, -9.44805, -9.44229, -9.44229, -9.44229, -9.44229, -9.44217, -9.44204, -9.44204, -9.44198, -9.44192, -9.44192, -9.44188, -9.44185, -9.44182, -9.44182, -9.44182, -9.4418, -9.44179, -9.44177, -9.44173, -9.44173, -9.44173, -9.44171, -9.44169, -9.44168, -9.44166, -9.44164, -9.44162, -9.44159, -9.44157, -9.44151, -9.44151, -9.44151, -9.44151, -9.4415, -9.44149, -9.44147, -9.44145, -9.44145, -9.44144, -9.44142, -9.44141, -9.44136, -9.44136, -9.44133, -9.4413, -9.44125, -9.4412, -9.44115, -9.44109, -9.44109, -9.44109, -9.44108, -9.44107, -9.44106, -9.44105, -9.44104, -9.44103, -9.44103, -9.44102, -9.44101, -9.441, -9.44099, -9.44097, -9.44095, -9.44093, -9.44092, -9.4409, -9.44087, -9.44085, -9.44082, -9.44082, -9.44082, -9.44082, -9.44082, -9.44082, -9.44082, -9.44082, -9.44082, -9.44082, ]
Contour(x, y, 'P2D', gt)

Using matplotlib backend: Qt5Agg


## Distribution-to-Distribution NDT

In [9]:
x = [0, 10.0284, 10.0284, 10.0284, 10.0284, 10.0284, 10.0284, 10.0284, 10.1899, 10.1899, 10.1899, 10.1607, 10.1607, 10.1607, 10.1607, 10.1544, 10.1601, 10.1601, 10.1555, 10.1594, 10.1594, 10.1559, 10.158, 10.1587, 10.1587, 10.1587, 10.157, 10.1587, 10.157, 10.1589, 10.1568, 10.1583, 10.1581, 10.1598, 10.1598, 10.1564, 10.1588, 10.1588, 10.1588, 10.1588, 10.1588, 10.1588, 10.1588, 10.1588, 10.1588, 10.1588, ]
y = [0, -9.52049, -9.52049, -9.52049, -9.52049, -9.52049, -9.52049, -9.52049, -9.44805, -9.44805, -9.44805, -9.44229, -9.44229, -9.44229, -9.44229, -9.44217, -9.44204, -9.44204, -9.44198, -9.44192, -9.44192, -9.44188, -9.44185, -9.44182, -9.44182, -9.44182, -9.4418, -9.44179, -9.44178, -9.44176, -9.44175, -9.44173, -9.44172, -9.44167, -9.44167, -9.44164, -9.44161, -9.44161, -9.44161, -9.44161, -9.44161, -9.44161, -9.44161, -9.44161, -9.44161, -9.44161, ]
Contour(x, y, 'D2D', gt)

Using matplotlib backend: Qt5Agg


## Symmetric NDT V1

In [10]:
x = [0, -9.36937, -79.4273, -79.4273, -79.4273, -79.4273, -79.4273, -79.4273, -79.4273, -74.1218, -61.0472, -38.0072, -17.4257, -8.24699, -5.11317, -4.41493, -4.33879, -4.33555, -4.3355, -4.3355, -4.3355, ]
y = [0, -3.89468, -9.96628, -9.96628, -9.96628, -9.96628, -9.96628, -9.96628, -9.96628, -10.194, -10.8229, -12.3194, -14.8483, -17.466, -19.1038, -19.5895, -19.6466, -19.649, -19.649, -19.649, -19.649, ]
Contour(x, y, 'SNDT', gt)

Using matplotlib backend: Qt5Agg


## Symmetric NDT V2

In [11]:
x = [0, 10.8376, 10.8381, 10.8381, 10.8381, ]
y = [0, -8.18661, -8.18702, -8.18702, -8.18702, ]
Contour(x, y, 'SNDT2', gt)

Using matplotlib backend: Qt5Agg


## Nonmonotonic Step

In [12]:
x = [0, 13.7534, 13.7548, 13.7548, 13.7548, ]
y = [0, -5.98317, -5.98377, -5.98377, -5.98377, ]
Contour(x, y, 'ICP', gt)

Using matplotlib backend: Qt5Agg


In [13]:
x = [0, 10.8376, 10.8381, 10.8381, 10.8381, ]
y = [0, -8.18661, -8.18702, -8.18702, -8.18702, ]
Contour(x, y, 'NICP', gt)

Using matplotlib backend: Qt5Agg


In [14]:
x = [0, 10.8376, 10.8381, 10.8381, 10.8381, ]
y = [0, -8.18661, -8.18702, -8.18702, -8.18702, ]
Contour(x, y, 'SICP', gt)

Using matplotlib backend: Qt5Agg


In [15]:
x = [0, 10.0284, 10.0284, 14.3026, 14.3026, 13.6751, 13.6751, 13.7664, 13.7664, 13.7531, 13.7531, 13.755, 13.755, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, ]
y = [0, -9.52049, -9.52049, -5.46385, -5.46385, -6.05943, -6.05943, -5.97277, -5.97277, -5.98537, -5.98537, -5.98354, -5.98354, -5.9838, -5.9838, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, ]
Contour(x, y, 'P2D', gt)

Using matplotlib backend: Qt5Agg


In [16]:
x = [0, 10.0284, 10.0284, 14.3026, 14.3026, 13.6751, 13.6751, 13.7664, 13.7664, 13.7531, 13.7531, 13.755, 13.755, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, 13.7548, ]
y = [0, -9.52049, -9.52049, -5.46385, -5.46385, -6.05943, -6.05943, -5.97277, -5.97277, -5.98537, -5.98537, -5.98354, -5.98354, -5.9838, -5.9838, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, -5.98377, ]
Contour(x, y, 'D2D', gt)

Using matplotlib backend: Qt5Agg


In [17]:
x = [0, -9.36937, -79.4273, -79.4273, -79.4273, 1809.08, 1664.49, 1668.51, 1668.51, 1668.51, ]
y = [0, -3.89468, -9.96628, -9.96628, -9.96628, 1153.67, 1247.06, 1244.01, 1244.01, 1244.01, ]
Contour(x, y, 'SNDT', gt)

Using matplotlib backend: Qt5Agg


In [18]:
x = [0, 10.8376, 10.8381, 10.8381, 10.8381, ]
y = [0, -8.18661, -8.18702, -8.18702, -8.18702, ]
Contour(x, y, 'SNDT2', gt)

Using matplotlib backend: Qt5Agg
